## Angular Distribution Analysis Using Dump DY

 To see the points in the histograms interactively [CLICK HERE](https://nbviewer.org/github/forhadnmsu/DumpAna/blob/main/dumpAna.ipynb)

Even after subtracting the combinatorial background (signal =unmixed events - mixed events), we deal with a large number of backgrounds. By finding the optimum event selections, we can remove the unphysical dimuons from the signal+other Background events. To take care of that, we can compare Monte Carlo distributions and find optimum selections. This is called signal to background significance $\frac{S}{\sqrt{S+B}}$. The details study can be found in the [docdb:10221](https://seaquest-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=10221). From the study, the track and dimuon event selections have been added to the ["cutDUMPDY.h"](https://github.com/forhadnmsu/DumpAna/blob/main/cutDUMPDY.h). The reconstructed events from the SeaQuest roadset-67 dataset and the reconstructed data from the Monte Carlo will get affected by the same even selections. All the seelctions will pass to the [RDataFrame Filter](https://root.cern/doc/master/classROOT_1_1RDataFrame.html). In the Filter, we have a lot selections that are added as && condition, and they are executed in order and the first one returning false causes the event to be discarded.

In [1]:
#include "TH1.h"
#include "TH2.h"
#include "TH3.h"
#include "TFile.h"
#include "TTree.h"
#include "TROOT.h"
#include "TStyle.h"
#include "TCanvas.h"
#include "TCut.h"
#include "TLegend.h"
#include "TLine.h"
#include "TLatex.h"
#include "TMarker.h"
#include "ana_const.h"
#include "cutDUMPDY.h"

The static function [SetDefaultSumw2()](https://root.cern/doc/master/classTH1.html#a8cd3b44217c8fec1d74046ed2ac153c2) is called in the beginning since we are going to deal with the weighted Histograms. Therefore the Sumw2() function is automatically called whenever any histograms is created.

In [4]:
%jsroot on
ROOT::EnableImplicitMT();
TH1::SetDefaultSumw2();
TH2::SetDefaultSumw2();
gStyle->SetPalette(1);
gStyle->SetPaintTextFormat("5.4f");
gROOT->ForceStyle();
gStyle->SetOptStat(0);
gStyle->SetOptFit(1111);
ROOT::RDataFrame dmyDf_mc("result_mc", "combined_LD2_DUMP_67_GMC.root");  //LD2 random hit-embedded MC
ROOT::RDataFrame dmyDf_data("result", "merged_RS67_3087all_target.root");  //All Target combined "unmixed" reconstructed toadset-67 events 
ROOT::RDataFrame dmyDf_mix("result_mix", "merged_RS67_3087all_target.root");//All Target combined "mixed" reconstructed toadset-67 events 

TH1D* h_pt = new TH1D("h_pt","h_pt", nBinpt, ptMin, ptMax);  // for pT Binning
TH1D* h_x2 = new TH1D("h_x2","h_x2", nBinx2, x2Min, x2Max);  // for xT binning
TCanvas* c1 = new TCanvas("c1","c1",1200,1600);
c1->Divide(2,4);
TFile *myfile = TFile::Open("acc_factor.root","read");
TF2* fit2D = new TF2("fit2D", "[0] * ( 1 + [1]*y*y + 2*[2]*sqrt(1-y*y)*y*cos(x) + [3]*(1-y*y)*cos(2*x)/2.) ", -M_PI, M_PI, -0.5, 0.5);
fit2D->SetParameters(1,1,0,0);
fit2D->SetParNames("A", "#lambda","#mu","#nu");

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


Angular distribution coefficients in the following conditions: 

1. Data: All Mixed target data.
2. GMC : LD2 random hit embedded Drell--Yan [docdb:9886](https://seaquest-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=9886) 
3. D1 < 110
4. 4 Dimuon pT bins in the range (0 to 2.0 GeV)
       

In [5]:
TCanvas* c1 = new TCanvas("c1","c1",800,1000);
c1->Divide(2,4);
for (int i_xT = 1; i_xT <=nBinx2 ; i_xT++) {
    for (int i_pt = 1; i_pt <=nBinpt ; i_pt++) {
        Double_t xBinMinPt = h_pt->GetXaxis()->GetBinLowEdge(i_pt);
        Double_t xBinMaxPt = h_pt->GetXaxis()->GetBinUpEdge(i_pt);
        Double_t xBinMinx2 = h_x2->GetXaxis()->GetBinLowEdge(i_xT);
        Double_t xBinMax2 = h_x2->GetXaxis()->GetBinUpEdge(i_xT);
        //RDATAFRAME
        string  ptCutTrue = Form("true_pT>%4.10f&&true_pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string trigCut ="fpga1==1&&";
        string massCut = "true_mass>5.0 && true_mass<8.0";
        string filter_mc_4pi=ptCutTrue+massCut; // 4 pi mc filter 
        string filter_mc_reco=ptCutTrue+trigCut+massCut+"&&"+dumpCutRD; // reco mc filter
        //Data
        string pTBins= Form("pT>%4.10f&&pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string xTBins=  Form("xT>%4.10f&&xT<%4.10f&&",xBinMinx2,xBinMax2);
        string massRange=" mass>5.0 && mass<8.0&&";
        string final_cut_filter=pTBins+xTBins+massRange+dumpCutRD; // data filter 

        auto hist2d_unmix = dmyDf_data.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_unmix", "phi_theta_unmix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax}, "phi_", "costh_");
        auto hist2d_mix = dmyDf_mix.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_mix", "phi_theta_mix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_", "costh_");

        TH2D* acc_factor = (TH2D*)myfile->Get(Form("h2_phi_costh_ixT_%i_ipt_%i",i_xT ,i_pt));

        hist2d_unmix->Add(hist2d_mix.GetPtr(),-1);
        cout <<"Integrated signal count: "<< hist2d_unmix->Integral()<<endl;
        hist2d_unmix->SetName(Form("phi_theta_ipT%i",i_pt));
        TH2*h2_phi_costh = (TH2D*)hist2d_unmix->Clone();

        int can = (2*i_pt-1);
        c1->cd(can);
        h2_phi_costh->Draw("colz text");
        h2_phi_costh->SetTitle(Form("(Unmix -Mix) Events in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        can++;
        c1->cd(can);
        TH2*acc_corrected_h2d = (TH2D*)h2_phi_costh->Clone();
        acc_corrected_h2d->Divide(acc_factor);
        acc_corrected_h2d->Fit("fit2D");
        cout << fit2D->GetChisquare() / fit2D->GetNDF() << endl;
        acc_corrected_h2d->SetTitle(Form("Acc. corrected in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        acc_corrected_h2d->Draw("colz");
        if(can==8){c1->Draw(); c1->SaveAs("acc_corrected.pdf");}
        delete acc_factor;
    }
}


Integrated signal count: 112776
 FCN=81.1154 FROM MIGRAD    STATUS=CONVERGED     878 CALLS         879 TOTAL
                     EDM=4.41188e-10    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.9 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            2.48817e+05   6.12622e+03   4.52452e+00   8.54640e-09
   2  #lambda      1.27515e+00   3.32216e-01  -3.07804e-04   1.40041e-04
   3  #mu          1.24518e-01   6.46555e-02   6.85341e-05   6.41183e-05
   4  #nu          9.15893e-02   5.55027e-02   4.50403e-06   2.35494e-05
1.35192
Integrated signal count: 176926
 FCN=160.728 FROM MIGRAD    STATUS=CONVERGED     110 CALLS         111 TOTAL
                     EDM=1.25983e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            4.

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file acc_corrected.pdf has been created


Figure Details: The left plots are the 2D angular distributions in the lower occupancy (D1<100) regions. The right plots are the acceptance corrected result. Right now we are only focusing on the expected statistical accuracy in the lower regions. The $\chi^2/NDF$ in the higher $pT$ needs more investigations.